<a href="https://colab.research.google.com/github/lotrmay/TensorFlow_Learning/blob/master/Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

t=tf.zeros([2,2,2,2]) #vytvoří tensor, zaplněn nulami s dimenzemi v závorkách
t=tf.reshape(t,[2,-1]) #přeskupí vybraný tensor, na jiný tensor, s jinými dimenzemi, které si určíme, můžeme si také dimenzi dopočítat zadáním -1
print(t)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np #better arrays in python, lepší práce s multidimenzionálními poli
import pandas as pd #data analytics tool, lepší manipulace s daty, dokáže například cut outnout column
import matplotlib.pyplot as plt #vizualizace tabulek a grafů
from IPython.display import clear_output #jen pro tenhle notebook
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc #

import tensorflow as tf


# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
#print(dftrain.head()) #vypíše nám prvních 5 záznamů a hlavičky dat
#print(dftrain["age"]) #vypíše sloupec age
y_train = dftrain.pop('survived') #vyjme z dat sloupec survived a uloží ho do y_train
y_eval = dfeval.pop('survived')
#print(dftrain.loc[0])#vypíše 1. řádek dat
#dftrain.describe() je metoda vhodná pro statistiky, dává nám údaje o datech jako vážený průměr, počet, min, max atd.
#dftrain.shape nám vrátí počet řádků a počet sloupců dat
#print(dftrain.age.hist(bins=20)) #vytvoří histogram, který reprezentuje data
#print(dftrain.sex.value_counts().plot(kind='barh')) vytvoří graf zobrazujicí počet pohlaví
#pd.concat([dftrain,y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')# graf, jenž nám zobrazí jakou má které pohlaví šanci na přežití

CATEGORICAL_COLUMNS=['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
NUMERIC_COLUMNS=['age','fare']

feature_columns=[]
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary=dftrain[feature_name].unique() #projede všechny řádky a zjistí každou unikátní hodnotu
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))

#print(feature_columns)

#následujicí kód nám poslouží abychom získali dataset object, kterému náš model bude rozumět
#epochs=kolikrát uvidí náš model data
#batch_size=Počet řádků, které uvidí každým průchodem, aby nedošlo k přehlcení paměti

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():#vnitřní funkce, která bude vrácena
    ds=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))# vytvoří tf.data.Dataset object a naplní ho daty 
    if shuffle: #slouží k tomu aby náš model nahlížel data i zjiné perspektivy
      ds=ds.shuffle(1000)#promíchá naše data
    ds=ds.batch(batch_size).repeat(num_epochs) # rozdělí dataset do 32 řádkových batchů a opakuje tolikrát kolik máme epochů
    return ds #vrátí nám batch datasetu
  return input_function # vrátí funkci pro náš objekt

train_input_fn=make_input_fn(dftrain,y_train) # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)#train
result=linear_est.evaluate(eval_input_fn)#výsledky toho, jak si model vedl

clear_output()#vyčistí konzoli
print(result['accuracy'])#vypíše nám výsledek, ale result obsahuje i například průměrný loss, jenž zde je 0.5, protože je to binární problém
#print(result)

#následujicí kód zkouší predikovat na základě vybraných dat

result=list(linear_est.predict(eval_input_fn))
print(dfeval.loc[5])
print(y_eval.loc[5])
print(result[5]['probabilities'][1])#index 1 nám zobrazí šanci, že přežil




In [ ]:
#Klasifikace 1:55:03
#https://www.youtube.com/watch?v=tPYj3fFJGjk

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np #better arrays in python, lepší práce s multidimenzionálními poli
import pandas as pd #data analytics tool, lepší manipulace s daty, dokáže například cut outnout column
import matplotlib.pyplot as plt #vizualizace tabulek a grafů
from IPython.display import clear_output #jen pro tenhle notebook
from six.moves import urllib


CSV_COLUMN_NAMES=['SpelaLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Setosa','Versicolor','Virginica']

train_path=tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path=tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train=pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test=pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
#načítání dat pomocí kerasu a panda
#print(train.head())
train_y=train.pop('Species')
test_y=test.pop('Species')

#train.head()#nyní jsou data bez Species
#train.shape #120 řádků a 4 sloupce

def input_fn(features,labels,training=True,batch_size=256):
  #Nyní vložíme naše data do datasetu
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  #Promíchej a opakuj pokud trénuješ
  if training:
    dataset=dataset.shuffle(1000).repeat()
  
  return dataset.batch(batch_size)

#Feature columns popisují jak použít náš input

my_feature_columns=[]
for key in train.keys():#train.keys() vrací všechny sloupce, mohli jsme loopovat přes CSV_COLUMN_NAMES, pokud bychom odebrali SPECIES v poli
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
#print(my_feature_columns)

classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns,hidden_units=[30,10], n_classes=3)
#n_classes říká počet tříd, a hiddent units znamená, že budeme mít 2 vrstvy a první bude mít 30 a druhá 10 nodes

classifier.train(input_fn=lambda: input_fn(train,train_y,training=True),steps=5000)#využití input funkce co jsme udělalli pro trénování
#lambda je v podstatě def funkce(): v jedné řádce
#steps je podobné jako epochs, steps označuje kolik záznamů musí model vidět pro ukončení


eval_result=classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))
#print('\nTest set accuracy:{accuracy:0.3f}\n'.format(**eval_result))

def input_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features=['SpelaLength','SepalWidth','PetalLength','PetalWidth']
predict={}

print("Please type numeric values as prompted.")

for feature in features:
  valid=True
  while valid:
    val=input(feature+": ")
    if not val.isdigit(): valid=False

  predict[feature]=[float(val)]

predictions=classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  class_id=pred_dict['class_ids'][0]
  probability=pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id],100*probability))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpu9hjyb9a', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:ten

KeyboardInterrupt: ignored

In [ ]:
#print(pred_dict['class_ids'][0]) #class_ids obsahuje predikovaný index rostliny v poli

2
